In [1]:
import os 
import numpy as np
from __future__ import print_function

import os
import argparse
import socket
import time
import sys

import torch
import torch.optim as optim
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader

import torchvision
from util import adjust_learning_rate, accuracy, AverageMeter
from torchvision import transforms
from torchvision.models import resnet50
from sklearn.metrics import f1_score

In [2]:
print(torch.__version__)

1.8.1+cu111


In [3]:
parser = argparse.ArgumentParser('argument for training')

parser.add_argument('--eval_freq', type=int, default=10, help='meta-eval frequency')
parser.add_argument('--print_freq', type=int, default=100, help='print frequency')
parser.add_argument('--tb_freq', type=int, default=500, help='tb frequency')
parser.add_argument('--save_freq', type=int, default=10, help='save frequency')
parser.add_argument('--batch_size', type=int, default=4, help='batch_size')
parser.add_argument('--num_workers', type=int, default=8, help='num of workers to use')
parser.add_argument('--epochs', type=int, default=200, help='number of training epochs')

# optimization
parser.add_argument('--learning_rate', type=float, default=0.0001, help='learning rate')
parser.add_argument('--lr_decay_epochs', type=str, default='60,80', help='where to decay lr, can be a list')
parser.add_argument('--lr_decay_rate', type=float, default=0.1, help='decay rate for learning rate')
parser.add_argument('--weight_decay', type=float, default=5e-4, help='weight decay')
parser.add_argument('--momentum', type=float, default=0.9, help='momentum')
parser.add_argument('--adam', action='store_true', help='use adam optimizer')

# dataset
parser.add_argument('--model', type=str, default='resnet50')

# cosine annealing
parser.add_argument('--cosine', action='store_true', help='using cosine annealing')

# specify folder
parser.add_argument('--model_path', type=str, default='', help='path to save model')
parser.add_argument('--tb_path', type=str, default='', help='path to tensorboard')
parser.add_argument('--data_root', type=str, default='', help='path to data root')

opt = parser.parse_args(args=[])

In [4]:
iterations = opt.lr_decay_epochs.split(',')
opt.lr_decay_epochs = list([])
for it in iterations:
    opt.lr_decay_epochs.append(int(it))

opt.n_gpu = torch.cuda.device_count()

In [5]:
# get datasets
train_transforms_option = transforms.Compose([
                    transforms.Resize((32, 32)),
                    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize([0.5071, 0.4867, 0.4408], [0.2675, 0.2565, 0.2761])
                ])
train_datasets = torchvision.datasets.ImageFolder(root="../../dataset/cifar100/train", transform = train_transforms_option)
train_loader = torch.utils.data.DataLoader(train_datasets, batch_size = 256, shuffle=True, num_workers = 4)


test_transforms_option = transforms.Compose([
                    transforms.Resize((32, 32)),
                    transforms.ToTensor(),
                    transforms.Normalize([0.5071, 0.4867, 0.4408], [0.2675, 0.2565, 0.2761])
                ])
test_datasets = torchvision.datasets.ImageFolder(root="../../dataset/cifar100/test", transform = test_transforms_option)
test_loader = torch.utils.data.DataLoader(test_datasets, batch_size = 256, shuffle=False, num_workers = 4)

In [ ]:
# model
n_cls = 100 # target에서 쓰는 class 수를 의미함
source_type = 'ours' #choice = ['random', 'shape', 'texture', 'ours']

if source_type == 'random' :
    print(f'We use {opt.model}')
    model = resnet50(pretrained = False)
    
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, n_cls)

elif source_type == 'shape' :
    opt.n_class = 10
    print(f'We use {opt.model}')
    model = resnet50(pretrained = False)
    
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, opt.n_class)

    model.load_state_dict(torch.load('../checkpoint_0302/shape_style1_best.pth'))
    
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, n_cls)
    
elif source_type == 'texture' :
    opt.n_class = 2
    print(f'We use {opt.model}')
    model = resnet50(pretrained = False)
    
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, opt.n_class)

    model.load_state_dict(torch.load('../checkpoint_0302/texture_style1_best.pth'))
    
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, n_cls)

elif source_type == 'ours' :
    opt.n_class = 20
    print(f'We use {opt.model}')
    model = resnet50(pretrained = False)
    
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, opt.n_class)

    model.load_state_dict(torch.load('../checkpoint_0302/ours_style1_best.pth'))
    
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, n_cls)

else :
    raise NotImplementedError(f'{source_type} is not implemented ..')

In [8]:
# optimizer
if opt.adam:
    optimizer = torch.optim.Adam(model.parameters(),lr=opt.learning_rate,weight_decay=0.0005)
else:
    optimizer = optim.SGD(model.parameters(),lr=opt.learning_rate,momentum=opt.momentum,weight_decay=opt.weight_decay)

criterion = nn.CrossEntropyLoss()

if torch.cuda.is_available():
    if opt.n_gpu > 1:
        model = nn.DataParallel(model)
    model = model.cuda()
    criterion = criterion.cuda()
    cudnn.benchmark = True

In [9]:
# set cosine annealing scheduler
if opt.cosine:
    eta_min = opt.learning_rate * (opt.lr_decay_rate ** 3)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, opt.epochs, eta_min, -1)

In [10]:
def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)

In [11]:
def train(epoch, train_loader, model, criterion, optimizer, opt):
    """One epoch training"""
    model.train()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    acc1 = AverageMeter()
    f1_1 = AverageMeter()
    
    total = 0
    correct = 0

    end = time.time()
    for idx, (input, target) in enumerate(train_loader):
        data_time.update(time.time() - end)

        input = input.float()
        if torch.cuda.is_available():
            input = input.cuda()
            target = target.cuda()

        # ===================forward=====================
        output = model(input)
        loss = criterion(output, target)
        acc = accuracy(output, target)
        pred = output.argmax(dim=1) # .view(output.shape)

        f1 = f1_score(target.cpu().detach().numpy(), pred.cpu().detach().numpy(), average = 'micro')
        losses.update(loss.item(), input.size(0))
        acc1.update(acc.item(), input.size(0))
        f1_1.update(f1.item(), input.size(0))

        # ===================backward=====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # ===================meters=====================
        batch_time.update(time.time() - end)
        end = time.time()

        # tensorboard logger
        pass

        # print info
        if idx % opt.print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'F1@1 {f1.val: 3f} ({f1.avg:3f})\t'.format(
                   epoch, idx, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=acc1, f1 = f1_1))
            sys.stdout.flush()

    print(' * Acc@1 {top1.avg:.3f} F1@1 {f1.avg:.3f}'.format(top1=acc1, f1=f1_1))

    return acc1.avg, losses.avg

In [12]:
def validate(val_loader, model, criterion, opt):
    """One epoch validation"""
    batch_time = AverageMeter()
    losses = AverageMeter()
    acc1 = AverageMeter()
    f1_1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for idx, (input, target) in enumerate(val_loader):

            input = input.float()
            if torch.cuda.is_available():
                input = input.cuda()
                target = target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)
            pred = output.argmax(dim=1) # .view(output.shape)
            f1 = f1_score(target.cpu().detach().numpy(), pred.cpu().detach().numpy(), average = 'micro')

            # measure accuracy and record loss
            acc = accuracy(output, target)
            losses.update(loss.item(), input.size(0))
            acc1.update(acc, input.size(0))
            f1_1.update(f1, input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if idx % opt.print_freq == 0:
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                      'F1@1 {f1.val:.3f} ({f1.avg:.3f})'.format(
                       idx, len(val_loader), batch_time=batch_time, loss=losses,
                       top1=acc1, f1=f1_1))

        print(' * Acc@1 {top1.avg:.3f}'
              .format(top1=acc1))

    return acc1.avg, losses.avg

In [ ]:
# 제대로 돌아가는 것까지 확인완료
best_acc = 0.0

for epoch in range(1, opt.epochs + 1):
    if opt.cosine:
        scheduler.step()
    else:
        adjust_learning_rate(epoch, opt, optimizer)
    print("==> training...")

    time1 = time.time()
    train_acc, train_loss = train(epoch, train_loader, model, criterion, optimizer, opt)
    time2 = time.time()
    print('epoch {}, total time {:.2f}'.format(epoch, time2 - time1))

    print(f'[Epoch{epoch}]train_acc', train_acc)
    print('train_loss', train_loss)

    test_acc, test_loss = validate(test_loader, model, criterion, opt)

    print(f'[epoch {epoch}] test_acc : {test_acc}')
    print(f'[epoch {epoch}] test_loss : {test_loss}')

    # regular saving
    if best_acc < test_acc :
        best_acc = test_acc
        print('==> Saving...')
        save_file = os.path.join('./checkpoint/best.pth'.format(epoch=epoch))
        torch.save(model.state_dict(), save_file)